<h1><b><center>Big Scale Analytics: Personal Assignment 2022</center></b></h1>

-----


**Contents**: the assignment covers four parts: 1. BigQuery, 2. Association rules, 3. Recommender Systems, 4. Elasticsearch.

**Due: Sunday April 10, 23.59pm** <u>(notebook + quiz)</u>

**Clarifications**: You can post your questions in slack channel #assignments. If necessary we will update the notebook accordingly (so make sure to check for updates on GitHub).

**Grading**: The personal assignment is worth **30%** of your final grade. For your work to be graded, you must:

* Upload your completed notebook on [Moodle](https://moodle.unil.ch/mod/assign/view.php?id=1264173)
* Answer all questions in this [Moodle quiz](https://moodle.unil.ch/mod/quiz/view.php?id=1264176). We will check that your quiz answers reflect the responses provided in the notebook.

>Note: You can only complete the quiz one time. Have your notebook with the answers ready for answering the quiz.

**Personal work**: Note that this assignment represents strictly *personal* work. Do not share it with your colleagues. Just do as much as you can on your own. Your code will be compared to that your colleagues. In case of statistically high similarity, you will receive a grade of zero.

**IMPORTANT**: In week 4, we will introduce ElasticSearch. Because your trial account will be limited to 14 days, we strongly advise you to start and finish Part Elastic Search before your account expires).

Good Luck and Enjoy ☀

----

## Part 1: SQL in BigQuery

In this first part, you will explore a dataset using Google BigQuery. Similar to week 2, you will connect to BigQuery, upload the data and access it in the notebook. Your job is to write SQL queries to answer the questions below.

The dataset is provided for you in the data folder: the `ratings.csv` and `movies.csv` files from the MovieLens 20M dataset.

**Connecting to BigQuery**

To make things easier, we advise you to work in **Google Colab**.

**For Google Colab users**

In [ ]:
from google.colab import auth

auth.authenticate_user()
print("Authenticated")

Authenticated


**For Jupyter users**

Make sure to replace "PATH_TO_CREDENTIALS_FILE" with the *absolute* path to the JSON service account key, e.g., "C:/Users/John/credentials.json".

In [ ]:
#!pip install google-cloud-bigquery

In [ ]:
#import os

#os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "PATH_TO_CREDENTIALS_FILE"

**Everyone**

Make sure to replace "YOUR-PROJECT-ID" with the ID of one of your Google Cloud projects, where you upload the data from the dats folder of the assignment.

In [ ]:
import pandas as pd
from google.cloud import bigquery

# Create a "Client" object
client = bigquery.Client(project="assignment-343016")

**The Dataset: Movie Lens 20M**
![movie_lens](https://bobliu.io/assets/img/homepage.13c535eb.jpg)

[The movie lens 20M dataset](https://grouplens.org/datasets/movielens/20m/) contains people's expressed preferences (ratings) for movies. These preferences, 20 million of them, were entered through the Movielens website - a recommender-system based business that asks its users to rate movies in order to receive personalized recommendations. No demographic information is included in the dataset. Each user is represented by an id, and no other information is provided.

We use the same dataset in week 6 of the lab for the introduction to recommender systems. To reduce the size of the ratings file, the original 20 million ratings dataset was trucated to some 40'000 ratings.

The **ratings** table provides information on the actual ratings, user id's and movie id.

The **movies** table provides more information on the movies themselves, movied ID, title, genres.

You can see below a preview of the two tables.


**Caution**: Do not forget that each user receives 1 TB of free data processing every month in BigQuery, which can be used to run queries on any dataset. Given the size of this dataset, performing a lot of queries can result in exceeding your free monthly quota. Therefore, you should try to avoid queries that have a big output. Always remember to use the **LIMIT** keyword (especially if you are not sure about the output of your query) to limit the size of the output.

The code below allows us to fetch and display the **schema** of the `movies` table in the movielens20M dataset.

In [ ]:
# Create a reference to the dataset
movielens_ref = client.dataset("data", project="assignment-343016")

# API request - fetch the dataset
movielens_dataset = client.get_dataset(movielens_ref)

# Create a reference to the "movies" table
movies_ref = movielens_ref.table("movies")

# Fetch the table (API request)
movies_table = client.get_table(movies_ref)

# Display schema
movies_table.schema

[SchemaField('movieId', 'INTEGER', 'NULLABLE', None, ()),
 SchemaField('title', 'STRING', 'NULLABLE', None, ()),
 SchemaField('genres', 'STRING', 'NULLABLE', None, ())]

In [ ]:
# List the tables in the dataset
tables = list(client.list_tables(movielens_dataset))
for table in tables:
    print(table.table_id)

movies
ratings


Showing a preview of the movies table using pandas.

In [ ]:
client.list_rows(movies_table, max_results=5).to_dataframe()

,movieId,title,genres
0,777,Pharaoh's Army (1995),War
1,966,"Walk in the Sun, A (1945)",War
2,1450,Prisoner of the Mountains (Kavkazsky plennik) ...,War
3,2669,Pork Chop Hill (1959),War
4,2670,Run Silent Run Deep (1958),War


Do the same for the ratings table.

In [ ]:
# Create a reference to the "ratings" table
ratings_ref = movielens_ref.table("ratings")

# Fetch the table (API request)
ratings_table = client.get_table(ratings_ref)

# Display schema
ratings_table.schema

[SchemaField('userId', 'INTEGER', 'NULLABLE', None, ()),
 SchemaField('movieId', 'INTEGER', 'NULLABLE', None, ()),
 SchemaField('rating', 'FLOAT', 'NULLABLE', None, ()),
 SchemaField('timestamp', 'INTEGER', 'NULLABLE', None, ())]

In [ ]:
#Show a preview of the "ratings" table using pandas
client.list_rows(ratings_table, max_results=5).to_dataframe()

,userId,movieId,rating,timestamp
0,2,891,2.0,974820969
1,2,1969,2.0,974820969
2,2,1970,2.0,974820969
3,2,1971,2.0,974820943
4,2,1972,2.0,974820969


### Question 1. What is the most common rating for films in the dataset ?
**Hint:** list the ratings and the number of films which received each rating.

In [ ]:
q1 = """
SELECT rating, count(movieId) AS sum_movies
FROM assignment-343016.data.ratings
GROUP BY rating
ORDER BY sum_movies DESC
LIMIT 1
"""

query_job = client.query(q1)
query_job.to_dataframe()

,rating,sum_movies
0,4.0,1179174


Answer Q1: 4.0

### Question 2. Find the title of the Comedy, with more than 1000 reviews, that received the highest average rating

**Hint**: For this question you will need to join the `ratings` and the `movies` tables. List the title, number of ratings, average rating. Look at movies which include `Comedy` in their genres description. Group by title (not by movieID).

In [ ]:
q2 = """
SELECT movies.title, COUNT(ratings.rating) as number_of_ratings, AVG(ratings.rating) as avg_rating
FROM  assignment-343016.data.movies movies
JOIN assignment-343016.data.ratings ratings
ON movies.movieId = ratings.movieId
WHERE movies.genres LIKE "%Comedy%"
GROUP BY movies.title
HAVING number_of_ratings > 1000
ORDER BY avg_rating DESC
LIMIT 1
"""

query_job = client.query(q2)
query_job.to_dataframe()

,title,number_of_ratings,avg_rating
0,Dr. Strangelove or: How I Learned to Stop Worr...,6768,4.237441


Answer Q2: Dr. Strangelove or: ...

### Question 3. Which user (userID) rated most romance films according to the available dataset and what was the average score he or she gave to all the romance films reviewed?

**Hint**: For this question you will need to join the ratings and the movies tables. List the userID, number of ratings of romance films, average rating of the romance films. Look at movies which include Romance in their genres description.

In [ ]:
q3 = """

SELECT  ratings.UserId, count(ratings.rating) AS number_of_ratings, avg(ratings.rating) AS average_rating
FROM  assignment-343016.data.ratings ratings
JOIN assignment-343016.data.movies movies
ON movies.movieId = ratings.movieId
WHERE movies.genres LIKE "%Romance%"
GROUP BY UserId
ORDER BY number_of_ratings DESC

LIMIT 1
"""

query_job = client.query(q3)
query_job.to_dataframe()

,UserId,number_of_ratings,average_rating
0,17455,468,2.655983


Answer Q3: UserId: 17455 ; average_rating: 2.655983

### Question 4.  Looking at the movies table in the available dataset, find the year in which most drama films were made.

**Hint:** You can find the year when a film was made at the end of most title strings, as for example in 'Waiting for Guffman (1996)', 1996 is the year shown in parenthesis at the end of the string, for most films. You can ignore the ones for which the information is not available in the title. You can use the REGEXP_EXTRACT function from the [Big Query reference documentation for string functions](https://cloud.google.com/bigquery/docs) to extract the year. A useful condition is related to the `genres` column of the table which should contain "Drama". Additonal hint: you can test your regex expression with the help of https://regex101.com/.

In [ ]:
q4 = """
SELECT
  (REGEXP_EXTRACT(movies.title, r'[(][0-9]{4}[)]')) AS year,
  count(movies.title) AS number_of_films
FROM assignment-343016.data.movies movies
WHERE movies.genres LIKE "%Drama%"
GROUP BY year
ORDER BY number_of_films DESC

LIMIT 5
"""
query_job = client.query(q4)
query_job.to_dataframe()

,year,number_of_films
0,(2009),508
1,(2007),475
2,(2011),471
3,(2008),465
4,(2006),446


Answer Q4: 2009

# Part 2: Association rules

In this second part, you will search for matches between film genres using the `mlxtend` Python module that we saw in week 5, by looking at the `movies.csv` dataset from the data folder of the assignment. Just to give an example: Drama and Action - how likely is it that a film from the dataset is both drama and action. This information can be interesting for film producers who may either want to produce something similar to the established norm: if most drama films are also action, perhaps the new action-drama film would be equally appreciated, or quite to the contrary try a new combination of genres which is more rare to find.

**Install required modules**

In [ ]:
!pip install mlxtend

Import the required packages and read in the data: the `assoc_rules_movies.csv` file from the data folder of the assignment. This is a modified version of the `movies.csv` file, to focus on the genre of each movie.

Each row represents a movie. The values in each row list the genres to which a movie belongs. In this dataset a movie belonged to a maximum of 10 genres. If a movie belongs to only 2 genres, say Drama and Action, the first column contains the first genre (Drama), the second contains the second genre (Action), and the following columns are filled with missing values. You do not need to do any data cleaning.

In [ ]:
import pandas as pd
import numpy as np
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules

data = pd.read_csv("https://raw.githubusercontent.com/michalis0/BigScaleAnalytics/master/assignment/data/assoc_rules_movies.csv")
data.head()

,0,1,2,3,4,5,6,7,8,9
0,Adventure,Animation,Children,Comedy,Fantasy,NaN,NaN,NaN,NaN,NaN
1,Adventure,Children,Fantasy,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Comedy,Romance,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Comedy,Drama,Romance,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Comedy,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Have a look at the unique values of genres from the first column.

In [ ]:
data.iloc[:,0].unique()

array(['Adventure', 'Comedy', 'Action', 'Drama', 'Crime', 'Children',
       'Mystery', 'Documentary', 'Animation', 'Thriller', 'Horror',
       'Fantasy', 'Western', 'Film-Noir', 'Romance', 'War', 'Sci-Fi',
       'Musical', 'IMAX'], dtype=object)

**Preprocessing**

As seen during the lab, convert the dataset using a `Transaction Encoder` from the `mlextend` module so that the dataset is reorganised in columns of unique genres (you should have 19 columns as there are 19 unique genres in the entire dataset). Each row should correspond to a different movie (27032 rows). Rows should contain only True or False boolean values according to whether a film was considered as belonging to a genre column or not.

In [ ]:
dataset = []
for ind, row in data.iterrows():
    transaction = []
    for item in row:
        if item == item:  # check if item is null
            transaction.append(item)
    dataset.append(transaction)

te = TransactionEncoder()
te_ary = te.fit(dataset).transform(dataset)
df = pd.DataFrame(te_ary, columns=te.columns_)
df.head()

,Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,False,True,True,True,True,False,False,False,True,False,False,False,False,False,False,False,False,False,False
1,False,True,False,True,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,True,False,False,False,False,False,False,False,False,False,True,False,False,False,False
3,False,False,False,False,True,False,False,True,False,False,False,False,False,False,True,False,False,False,False
4,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False


In [ ]:
df.shape

(27032, 19)

**Frequent itemsets**  
Using the apriori algorithm find the frequent itemsets with minimum support of 0.01. There is no condition on the maximum length of an itemset.

In [ ]:
freq_items = apriori(df, min_support=0.01, use_colnames=True)
freq_items.head(10)

,support,itemsets
0,0.130216,(Action)
1,0.086157,(Adventure)
2,0.037992,(Animation)
3,0.042135,(Children)
4,0.309781,(Comedy)
5,0.108723,(Crime)
6,0.091410,(Documentary)
7,0.493637,(Drama)
8,0.052234,(Fantasy)
9,0.012208,(Film-Noir)


### Question 5. How many itemsets did the apriori algorithm return above (for min_support=0.01)?
Count in all retrieved itemsets, regardless of the number of elements (no condition on the maximum or minimum length of an itemset).

In [ ]:
freq_items.shape

(69, 2)

Answer: 69

**Mining for association rules**  
Using the frequent items identified above, find association rules with a minimum confidence of 0.5 and order them by decreasing value of lift.

In [ ]:
rules = association_rules(freq_items, metric="confidence", min_threshold=0.5)
rules.head(15).sort_values(by="lift", ascending=False)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
3,(Mystery),(Thriller),0.056008,0.154558,0.029410,0.525099,3.397434,0.020753,1.780250
4,"(Action, Crime)",(Thriller),0.028374,0.154558,0.014612,0.514993,3.332050,0.010227,1.743157
2,(War),(Drama),0.044170,0.493637,0.033072,0.748744,1.516790,0.011268,2.015324
1,(Romance),(Drama),0.152671,0.493637,0.095183,0.623455,1.262983,0.019819,1.344762
0,(Crime),(Drama),0.108723,0.493637,0.063332,0.582511,1.180039,0.009663,1.212877
5,"(Thriller, Crime)",(Drama),0.045576,0.493637,0.025192,0.552760,1.119769,0.002695,1.132194


### Question 6. Looking at your generated association rules, which of the following statements are true of films in our dataset?
a. Mystery films are associated with thrillers  
b. If a film is a drama, then it is also a romance film  
c. If war then drama is the asociation rule with the highest confidence  
d. Adventure-comedy-action occur more frequently than expected  
e. None of the above

a, c, ...

----

# Part 3: Recommender Systems

Using the surprise library, with item-based collaborative filtering, find the top 10 recommended films to watch for a given user, similar to what we have seen in the lab. [Documentation about surprise is available here](http://surpriselib.com/).

We will use the 100k MovieLens dataset, smaller as compared to what we have used before. Follow the guiding steps below and answer the two questions.

**First install the library and import the required packages**

In [ ]:
!pip install surprise

     |████████████████████████████████| 11.8 MB 4.5 MB/s 
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.1-cp37-cp37m-linux_x86_64.whl size=1630160 sha256=80a8541efab07dd279ac3c041a8976f7c24910b7cc8cbb2bb13f68bf4fb6262d
  Stored in directory: /root/.cache/pip/wheels/76/44/74/b498c42be47b2406bd27994e16c5188e337c657025ab400c1c
Successfully built scikit-surprise


In [ ]:
# Import packages
from surprise import KNNBasic, KNNWithMeans
from surprise import Dataset
from surprise.model_selection import GridSearchCV
from collections import defaultdict
from surprise import get_dataset_dir
from surprise.model_selection import train_test_split
import io

**Load the data using the built-in dataset `ml-100k`**

In [ ]:
# Load data
data = Dataset.load_builtin('ml-100k')

Dataset ml-100k could not be found. Do you want to download it? [Y/n] Y
Trying to download dataset from http://files.grouplens.org/datasets/movielens/ml-100k.zip...
Done! Dataset ml-100k has been saved to /root/.surprise_data/ml-100k


**Use GridSearchCV to find the best number of neighbours (k) for a KNNWithMeans item-based algorithm**:  
using root-mean-square-error (RMSE) and the following parameter grid `param_grid={'k': [20, 30, 40, 50],                                                      'sim_options': {'name': ['pearson'], 'user_based': [False]}}`  
other parameters `cv=5, refit=True, joblib_verbose=2, n_jobs=-1`

In [ ]:
KNN_grid_search = GridSearchCV(KNNWithMeans, param_grid={'k': [20, 30, 40, 50],
                                                         'sim_options': {'name': ['pearson'], 'user_based': [False]}},
                               measures=['RMSE'], cv=5,
                               refit=True, joblib_verbose=2, n_jobs=-1)
KNN_grid_search.fit(data)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  20 out of  20 | elapsed:  2.2min finished


Computing the pearson similarity matrix...
Done computing similarity matrix.


In [ ]:
print("best parameter:", KNN_grid_search.best_params)
print("best rmse: ", KNN_grid_search.best_score)
# you can even see the whole cv results
print("\n")
KNN_grid_search.cv_results

best parameter: {'rmse': {'k': 50, 'sim_options': {'name': 'pearson', 'user_based': False}}}
best rmse:  {'rmse': 0.9405360955094061}




{'mean_fit_time': array([5.24030871, 5.89345198, 5.46694779, 5.03225217]),
 'mean_test_rmse': array([0.95420753, 0.94611834, 0.9424826 , 0.9405361 ]),
 'mean_test_time': array([6.39515467, 6.89635673, 7.12449703, 6.37281642]),
 'param_k': [20, 30, 40, 50],
 'param_sim_options': [{'name': 'pearson', 'user_based': False},
  {'name': 'pearson', 'user_based': False},
  {'name': 'pearson', 'user_based': False},
  {'name': 'pearson', 'user_based': False}],
 'params': [{'k': 20, 'sim_options': {'name': 'pearson', 'user_based': False}},
  {'k': 30, 'sim_options': {'name': 'pearson', 'user_based': False}},
  {'k': 40, 'sim_options': {'name': 'pearson', 'user_based': False}},
  {'k': 50, 'sim_options': {'name': 'pearson', 'user_based': False}}],
 'rank_test_rmse': array([4, 3, 2, 1]),
 'split0_test_rmse': array([0.95334135, 0.94485396, 0.94093796, 0.93908426]),
 'split1_test_rmse': array([0.95778632, 0.9495039 , 0.94533529, 0.94317355]),
 'split2_test_rmse': array([0.9525417 , 0.94430033, 0.9411

### Question 7. What is the optimal k for which GridSearchCV returned the best RMSE score?

Answer Q7 : k = 50

**Use the full dataset and the `build_full_trainset()` method to build a trainset object. Then use the `build_anti_testset()` method for building the trainset.**

In [ ]:
trainset = data.build_full_trainset()
testset = trainset.build_anti_testset()

**Instantiate the KNNWithMeans algorithm using the best k value retrieved above: `KNNWithMeans(k=YOUR_RETRIEVED_VALUE, min_k=1, sim_options=sim_options, verbose=False)`**  
KNNWithMeans takes into account the mean ratings of each user. You can read more about it here: [Documentation](https://surprise.readthedocs.io/en/stable/knn_inspired.html)

In [ ]:
sim_options = {
    'name': 'pearson',
    'user_based': False #it is not specified if we have to take True (like in the lab) or False (like in previous question) for the boolean value of user_based
}
knn_means = KNNWithMeans(k=50, min_k=1, sim_options=sim_options, verbose=False)

**Fit the model on the training set and predict ratings on the test set.**

In [ ]:
# Fit model
knn_means.fit(trainset)
predictions = knn_means.test(testset)

**Use the helper function below to identify the best 20 films for all users**

In [ ]:
def read_item_names():
    '''Read the u.item file from MovieLens 100-k dataset and return two
    mappings to convert raw ids into movie names and movie names into raw ids.
    '''

    file_name = get_dataset_dir() + '/ml-100k/ml-100k/u.item'
    rid_to_name = {}
    name_to_rid = {}
    with io.open(file_name, 'r', encoding='ISO-8859-1') as f:
        for line in f:
            line = line.split('|')
            rid_to_name[line[0]] = line[1]
            name_to_rid[line[1]] = line[0]

    return rid_to_name, name_to_rid


def get_top_n(predictions, n=20):
    '''Return the top-N recommendation for each user from a set of predictions.

    Args:
        predictions(list of Prediction objects): The list of predictions, as
            returned by the test method of an algorithm.
        n(int): The number of recommendation to output for each user. Default
            is 10.

    Returns:
    A dict where keys are user (raw) ids and values are lists of tuples:
        [(raw item id, rating estimation), ...] of size n.
    '''

    # First map the predictions to each user.
    top_n = defaultdict(list) # This is used to group a sequence of key-value pairs into a dictionary of lists
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))

    # Then sort the predictions for each user and retrieve the k highest ones.
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]

    return top_n

**Find the top 20 predictions for user 196.**

In [ ]:
top_n = get_top_n(predictions, n=20)
top_n['196']

[('1189', 5),
 ('1500', 5),
 ('814', 5),
 ('1125', 5),
 ('1536', 5),
 ('1293', 5),
 ('1191', 5),
 ('1243', 5),
 ('1599', 5),
 ('1653', 5),
 ('1467', 5),
 ('1122', 5),
 ('1201', 5),
 ('1398', 4.886939234410358),
 ('1512', 4.811371461209678),
 ('134', 4.671506337126496),
 ('1639', 4.6170012366381865),
 ('114', 4.6077539200628825),
 ('64', 4.605529315055976),
 ('701', 4.600410624902662)]

### Question 8. Looking at the top 20 predictions for user 196 above, select the correct statements below:
a. MovieId 1189 is predicted to receive rating 5    
b. MovieId 134 is predicted to receive rating 3    
c. MovieId 814 is predicted to receive rating 4   
d. MovieId 1500 is predicted to receive rating 5  
e. None of the above

a.
d.

----

# Part 4: Elasticsearch

In this part, you will use Elasticsearch's JSON-based Query DSL in order to perform analysis on a dataset of approximately 2'250 films tagged by some nearly 200 users, a sample from the same MovieLent20M database, which you can find in the data folder of the assignment as `tagged_films.csv`.

**NB**: In week 4, we will introduce ElasticSearch. Because your trial account will be limited to 14 days, we strongly advise you to start and finish this part between weeks 4 and 5).

**Apache Access Log**

In a real-world scenario, you could set up a live feed from the system of interest, where you log movie tags to the Elastic cloud. Here, for simplicity's sake, you will upload the data as a file, which we will also do during the lab as an exercise.

The data contains movie tags, such as "true story", "classic", "thought provoking", together with moveIds, userIds, film titles, genres information, as well as the date of the tag.

In the file, each line corresponds to a tag. Here is an example of an entry in this table:

{
        "_index" : "tagged_movies",
        "_id" : "9QcsGH8BJ2RbyKf7cYWF",
        "_score" : 1.0,
        "_source" : {
          "@timestamp" : "2014-04-13T05:33:00.000+02:00",
          "genres" : "Drama",
          "movieId" : 7158,
          "tag" : "sad but good",
          "title" : "House of Sand and Fog (2003)",
          "userId" : 133,
          "timestamp" : "2014-04-13 05:33:00"
        }
      }

**Loading the Data to Elastic**

You should have already deployed an Elasticsearch service on the Elastic cloud during the lab (if you haven't, please refer to the lab of week 4).

Once your deployment is ready, go to the **Kibana dashboard** and go through the following steps:

1. Download the tagged_movies.csv file from the data folder of the assignment.
2. From the dashboard's menu, choose "Upload file" and upload the file.
4. Once the upload is complete, you may need to override the pattern so that title rows are recognized.
  
5. Finally, click "Import" and specify an index name. Take note of the name you choose, as you will need it when writing queries. Also, make sure that the "Create index pattern" box is checked.

Once that process is complete, the data will be indexed and available for query.

**Answering Questions**

Please note:

* To run your queries, you must go to the **console** (from the side menu, *Management* > *Dev Tools*).
* The queries must be written in Elasticsearch's **Query DSL** (Domain Specific Language). The full documentation (along with code samples) can be found [here](https://www.elastic.co/guide/en/elasticsearch/reference/current/query-dsl.html).
* Once you have found the query to answer each of the four questions below, **copy/paste** the query in the notebook. See an example below.

In [ ]:
# EXAMPLE DSL QUERY
GET tagged_movies/_search
{
  "query": {
    "match_all": {}
  }
}


In [ ]:

# EXAMPLE DSL QUERY
GET data/_search
 {
    "query": {
        "range" : {
            "price" : {
                "gte" : 1000,
                "lte" : 2000
            }
        }
    }
}

**Alternatively**, you can connect your notebook to your ElasticSearch deployment and run the queries directly in the notebook.   
Guiding steps and an example are provided below.

In [ ]:
# first install the required package
!pip install elasticsearch-dsl

     |████████████████████████████████| 63 kB 1.8 MB/s 
     |████████████████████████████████| 385 kB 19.5 MB/s 


In [ ]:
import pandas as pd
import elasticsearch_dsl
from elasticsearch_dsl import connections
from elasticsearch import Elasticsearch
from elasticsearch_dsl import Search, Q, A

In [ ]:
# connecting to your elastic search deployment
# for the first entry between '' go to Elasticsearch Service > Deployment management> Copy Elasticsearch endpoint
# for the second entry between '' you will need to use the authentication credentials from when you set up your deployment
client = Elasticsearch('https://bsa.es.europe-west3.gcp.cloud.es.io:9243',
                      http_auth=('elastic','WEB1uKuJP9tZ1mvb0vfORPB2'))

INDEX NAME = DATA

Example of query below for retrieving all tags related to movieId 1111.

In [ ]:
# set index to the name under which you uploaded your data in your deployment
s = Search(using=client, index="data").query("match", movieId="1111")
response = s.execute()

In [ ]:
df = []
for h in response.hits.hits:
    df.append(h["_source"].to_dict())

pd.DataFrame(df)

,@timestamp,genres,movieId,tag,title,userId,timestamp
0,2008-08-13T14:03:16.000+02:00,Documentary,1111,animals,Microcosmos (Microcosmos: Le peuple de l'herbe...,1741,2008-08-13 14:03:16
1,2006-01-15T18:53:39.000+01:00,Documentary,1111,animals,Microcosmos (Microcosmos: Le peuple de l'herbe...,1763,2006-01-15 18:53:39
2,2006-01-15T18:53:39.000+01:00,Documentary,1111,insects,Microcosmos (Microcosmos: Le peuple de l'herbe...,1763,2006-01-15 18:53:39




### Question 9. Which film (movieId) received the largest number of tags between 5 March 2014 and 5 March 2015?

**Hint**: Use the following date format for writing your condition: [yyyy-MM-dd HH:mm:ss], and set the hours, minutes and seconds to 0. The limits of the time interval should be included (as in writing a condition for greater or equal to). Use `movieId` for the aggregation function (not the `title`).

In [ ]:
# COPY-PASTE OF MY ELASTIC DSL REQUEST

GET data/_search
{
  "query": {
    "range": {
      "timestamp": {
        "gte": "2014-03-05 00:00:00",
        "lte": "2015-03-05 00:00:00"
      }
    }
  },
  "aggs": {
    "by_movieID": {
      "terms": {
        "field": "movieId"
      }
    }
  }
}

Answer Q9:
movieId: 4973
number_of_tags: 28

In [ ]:
#| Q("match", timestamp : ["2015-03-05 00:00:00") #timestamp>="2014-03-05 00:00:00") | Q("match", timestamp <= "2015-03-05 00-00-00")

### Question 10. What was the most popular word used in tags published between 5 March 2010 and 5 March 2015?

**Hint**: Use the following date format for writing your condition: [yyyy-MM-dd HH:mm:ss], and set the hours, minutes and seconds to 0. The limits of the time interval should be included (as in writing a condition for greater or equal to). Use a [significant text aggregation](https://www.elastic.co/guide/en/elasticsearch/reference/current/search-aggregations-bucket-significanttext-aggregation.html) on tags to find the words most commonly used.

In [ ]:
# COPY-PASTE OF MY ELASTIC DSL REQUEST

GET data/_search
{
  "query": {
    "range": {
      "timestamp": {
        "gte": "2010-03-05 00:00:00",
        "lte": "2015-03-05 00:00:00"
      }
    }
  },
  "aggs": {
    "by_frequency_of_term_in_tags": {
      "significant_text": {
        "field": "tag"
      }
    }
  }
}

Answer Q10: "ending"

### Question 11. How many films produced in 2002 were tagged with "true story"?
**Hint**: Look at films whose titles mention the year (2002) and for which the tag matches "true story". Results can be aggregated by movieId to find the number of films.

In [ ]:
# COPY-PASTE OF MY ELASTIC DSL REQUEST

GET data/_search
{
  "query": {
    "bool": {
      "must": [
        {
        "match": {
          "title": "2002"
         }
        },
        {
          "match": {
            "tag": "true story"
          }
        }
      ]
    }
  },
  "aggs": {
    "by_movieID": {
      "terms": {
        "field": "movieId"
      }
    }
  }
}


Answer Q11: 4
Titles: The pianist, City of God, Gangs of New York, Catch Me If You Can


### Question 12. Which userID produced most of the tags in the dataset?

In [ ]:
# COPY-PASTE OF MY ELASTIC DSL REQUEST

GET data/_search
{
  "aggs": {
    "by_userId": {
      "terms": {
        "field": "userId"
      }
    }
  }
}

Answer Q12:
userId: 1741